# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win percentage's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win percentage's correlation with the population of the city it is in for the **NHL** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df = pd.read_csv("assets/nhl.csv")

    cities_df = pd.read_html("assets/wikipedia_data.html")[1]
    cities_df = cities_df.iloc[:-1,[0,3,5,6,7,8]]
    
    cities_df.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities_df = cities_df.astype({'Population': 'int64'}) 
    cities_df = cities_df.replace('(\[note\s\d{1,10}\])', '', regex=True)
    cities_df = cities_df.replace('—[\s]?', " ", regex=True)

    single_teams_df = cities_df['NHL'].str.replace( r"(?<!\s)(?=[A-Z])", '+', regex=True)
    single_teams_df= single_teams_df.str.lstrip('+')
    single_teams_df= single_teams_df.str.split('+')

    cities_df['NHL'] = single_teams_df
    cities_df = cities_df.drop(['NFL', 'MLB', 'NBA'], axis=1)
    cities_df = cities_df.explode('NHL').reset_index(drop=True)
    cities_df['NHL'] = cities_df['NHL'].str.split().str[-1]

    nhl_df = (nhl_df[nhl_df['year'] == 2018]
                  .replace('\*','',regex=True)[['team','W','L']][~nhl_df['team'].str.contains("Division")]).reset_index(drop=True)
    

    nhl_df = nhl_df.astype({'W': 'int64','L': 'int64' }) 
    nhl_df['W/L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    nhl_df.drop(['W','L'], inplace=True, axis=1) 
    nhl_df.rename(columns={'W/L%':'ratio'}, inplace=True)
    nhl_df['team'] = nhl_df['team'].str.replace(r'\*',"")
    nhl_df['team'] = nhl_df['team'].str.split().str[-1]

    combined_city_nhl_df = pd.merge(nhl_df, cities_df,
                                    how='inner', 
                                    left_on='team',
                                    right_on='NHL').groupby('Metropolitan area').agg({'ratio': np.mean, 'Population': np.mean})                                                                                                                 
    
    #combined_city_nhl_df = combined_city_nhl_df.groupby('Metropolitan area').agg({'ratio': np.mean, 'Population': np.mean})

    
    population_by_region = combined_city_nhl_df['Population'] 
    win_loss_by_region = combined_city_nhl_df['ratio']

    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return population_by_region, win_loss_by_region, correlation


nhl_correlation()

(Metropolitan area
 Boston                     4794447.0
 Buffalo                    1132804.0
 Calgary                    1392609.0
 Chicago                    9512999.0
 Columbus                   2041520.0
 Dallas–Fort Worth          7233323.0
 Denver                     2853077.0
 Detroit                    4297617.0
 Edmonton                   1321426.0
 Las Vegas                  2155664.0
 Los Angeles               13310447.0
 Miami–Fort Lauderdale      6066387.0
 Minneapolis–Saint Paul     3551036.0
 Montreal                   4098927.0
 Nashville                  1865298.0
 New York City             20153634.0
 Ottawa                     1323783.0
 Philadelphia               6070500.0
 Phoenix                    4661537.0
 Pittsburgh                 2342299.0
 Raleigh                    1302946.0
 San Francisco Bay Area     6657982.0
 St. Louis                  2807002.0
 Tampa Bay Area             3032171.0
 Toronto                    5928040.0
 Vancouver                  246

In [2]:
population_by_region, win_loss_by_region, correlation = nhl_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"


In [3]:
population_by_region, win_loss_by_region, correlation = nhl_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"


## Question 2
For this question, calculate the win percentage's correlation with the population of the city it is in for the **NBA** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():

    
    nba_df=pd.read_csv("assets/nba.csv")
    cities_df=pd.read_html("assets/wikipedia_data.html")[1]
    cities_df=cities_df.iloc[:-1,[0,3,5,6,7,8]]
    
    cities_df.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities_df = cities_df.astype({'Population': 'int64'}) 
    cities_df = cities_df.replace('(\[note\s\d{1,10}\])', '', regex=True)
    cities_df = cities_df.replace('—[\s]?', " ", regex=True)
    #display(cities_df)
    
    single_teams_df = cities_df['NBA'].str.replace( r"(?<!\s)(?=[A-Z])", '+', regex=True)
    single_teams_df= single_teams_df.str.lstrip('+')
    single_teams_df= single_teams_df.str.split('+')

    cities_df['NBA'] = single_teams_df
    cities_df = cities_df.drop(['NFL', 'MLB', 'NHL'], axis=1)
    cities_df = cities_df.explode('NBA').reset_index(drop=True)
    cities_df['NBA'] = cities_df['NBA'].str.split().str[-1]
    #display(cities_df)
    
    nba_df = (nba_df[nba_df['year'] == 2018]
                  .replace('\*','',regex=True)[['team','W','L']][~nba_df['team'].str.contains("Division")]).reset_index(drop=True)
    #display(nba_df)
    nba_df = nba_df.astype({'W': 'int64','L': 'int64' }) 
    nba_df['W/L%'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    nba_df.drop(['W','L'], inplace=True, axis=1) 
    nba_df.rename(columns={'W/L%':'ratio'}, inplace=True)
    nba_df['team'] = nba_df['team'].str.replace(r'\*',"")
    nba_df['team'] = nba_df['team'].str.split().str[-2]
 
    #display(nba_df)

    combined_city_nba_df = pd.merge(nba_df, cities_df,
                                    how='inner', 
                                    left_on='team',
                                    right_on='NBA').groupby('Metropolitan area').agg({'ratio': np.mean, 'Population': np.mean})                                      
    #display(combined_city_nhl_df)                                                                            
    
    #combined_city_nba_df = combined_city_nba_df.groupby('Metropolitan area').agg({'ratio': np.mean, 'Population': np.mean})
    #display(combined_city_nba_df)
    
    population_by_region = combined_city_nba_df['Population'] 
    win_loss_by_region = combined_city_nba_df['ratio']
    
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return population_by_region, win_loss_by_region, correlation

# YOUR CODE HERE
#raise NotImplementedError()

In [5]:
population_by_region, win_loss_by_region, correlation = nba_correlation()
assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"


## Question 3
For this question, calculate the win percentage's correlation with the population of the city it is in for the **MLB** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation():

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities_df=pd.read_html("assets/wikipedia_data.html")[1]
    cities_df=cities_df.iloc[:-1,[0,3,5,6,7,8]]
    
    cities_df.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities_df = cities_df.astype({'Population': 'int64'}) 
    cities_df = cities_df.replace('(\[note\s\d{1,10}\])', '', regex=True)
    cities_df = cities_df.replace('—[\s]?', " ", regex=True)
    #display(cities_df)
    
    single_teams_df = cities_df['MLB'].str.replace( r"(?<!\s)(?=[A-Z])", '+', regex=True)
    single_teams_df= single_teams_df.str.lstrip('+')
    single_teams_df= single_teams_df.str.split('+')
    #display(single_teams_df)

    cities_df['MLB'] = single_teams_df
    cities_df = cities_df.drop(['NFL', 'NBA', 'NHL'], axis=1)
    cities_df = cities_df.explode('MLB').reset_index(drop=True)
    cities_df['MLB'] = cities_df['MLB'].str.split().str[-1]
    #display(cities_df)
    
    mlb_df = (mlb_df[mlb_df['year'] == 2018]
                  .replace('\*','',regex=True)[['team','W','L']][~mlb_df['team'].str.contains("Division")]).reset_index(drop=True)
    #display(mlb_df)
    mlb_df = mlb_df.astype({'W': 'int64','L': 'int64' }) 
    mlb_df['W/L%'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    mlb_df.drop(['W','L'], inplace=True, axis=1) 
    mlb_df.rename(columns={'W/L%':'ratio'}, inplace=True)
    mlb_df['team'] = mlb_df['team'].str.replace(r'\*',"")
    mlb_df['team'] = mlb_df['team'].str.split().str[-1]
 
    #display(mlb_df)

    combined_city_mlb_df = pd.merge(mlb_df, cities_df,
                                    how='inner', 
                                    left_on='team',
                                    right_on='MLB')
    combined_city_mlb_df = combined_city_mlb_df.drop([0,3], axis=0).reset_index(drop=True)
    #display(combined_city_mlb_df)   #inner, team, mlb                                                                         
    
    combined_city_mlb_df = combined_city_mlb_df.groupby('Metropolitan area').agg({'ratio': np.nanmean, 'Population': np.nanmean})
    
    #display(combined_city_mlb_df)
    
    population_by_region = combined_city_mlb_df['Population'] 
    win_loss_by_region = combined_city_mlb_df['ratio']
    
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return population_by_region, win_loss_by_region, correlation

mlb_correlation()

(Metropolitan area
 Atlanta                    5789700.0
 Baltimore                  2798886.0
 Boston                     4794447.0
 Chicago                    9512999.0
 Cincinnati                 2165139.0
 Cleveland                  2055612.0
 Dallas–Fort Worth          7233323.0
 Denver                     2853077.0
 Detroit                    4297617.0
 Houston                    6772470.0
 Kansas City                2104509.0
 Los Angeles               13310447.0
 Miami–Fort Lauderdale      6066387.0
 Milwaukee                  1572482.0
 Minneapolis–Saint Paul     3551036.0
 New York City             20153634.0
 Philadelphia               6070500.0
 Phoenix                    4661537.0
 Pittsburgh                 2342299.0
 San Diego                  3317749.0
 San Francisco Bay Area     6657982.0
 Seattle                    3798902.0
 St. Louis                  2807002.0
 Tampa Bay Area             3032171.0
 Toronto                    5928040.0
 Washington, D.C.           613

In [7]:
population_by_region, win_loss_by_region, correlation = mlb_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"


## Question 4
For this question, calculate the win percentage's correlation with the population of the city it is in for the **NFL** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    #display(nfl_df)
    cities_df=pd.read_html("assets/wikipedia_data.html")[1]
    cities_df=cities_df.iloc[:-1,[0,3,5,6,7,8]]
    
    cities_df.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities_df = cities_df.astype({'Population': 'int64'}) 
    cities_df = cities_df.replace('(\[note\s\d{1,10}\])', '', regex=True)
    cities_df = cities_df.replace('—[\s]?', " ", regex=True)
   
    #display(cities_df)
    
    single_teams_df = cities_df['NFL'].str.replace( r"(?<!\s)(?=[A-Z])", '+', regex=True)
    single_teams_df= single_teams_df.str.lstrip('+')
    single_teams_df= single_teams_df.str.split('+')
    #display(single_teams_df)

    cities_df['NFL'] = single_teams_df
    cities_df = cities_df.drop(['MLB', 'NBA', 'NHL'], axis=1)
    cities_df = cities_df.explode('NFL').reset_index(drop=True)
    cities_df['NFL'] = cities_df['NFL'].str.split().str[-1]
    
    #cities_df['NFL']= cities_df['NFL'].dropna().reset_index(drop=True)
    #display(cities_df)
    #display(nfl_df)
    
    nfl_df = (nfl_df[nfl_df['year'] == 2018]
              .replace('\*','',regex=True)[['team','W','L']][~nfl_df['team'].str.contains("FC")]).reset_index(drop=True)
    
    #display(nfl_df)

    nfl_df = nfl_df.astype({'W': 'int64','L': 'int64' }) 
    nfl_df['W/L%'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    #nfl_df.drop(['W','L'], inplace=True, axis=1)
    nfl_df.rename(columns={'W/L%':'ratio'}, inplace=True)
    nfl_df['team'] = nfl_df['team'].str.replace(r'\*',"")
    nfl_df['team'] = nfl_df['team'].str.split().str[-1]
    nfl_df['team'] = nfl_df['team'].str.strip('+')


    #display(cities_df)
    #display(nfl_df)
    
    combined_city_nfl_df = pd.merge(nfl_df, cities_df,
                                    how='inner', 
                                    left_on='team',
                                    right_on='NFL').groupby('Metropolitan area').agg({'ratio': np.mean, 'Population': np.mean})
 
    #nfl_df = nfl_df.drop([10,17,29], axis=0).reset_index(drop=True)
    #display(nfl_df)
    
    #combined_city_nfl_df = combined_city_nfl_df.drop([10,17,19], axis=0).reset_index(drop=True)
    
    #display(combined_city_nfl_df)
    
    #combined_city_nfl_df = combined_city_nfl_df.groupby('Metropolitan area').agg({'ratio': np.mean, 'Population': np.mean})
    #display(combined_city_nfl_df)
    #display(nfl_df)
    
    population_by_region = combined_city_nfl_df['Population'] 
    win_loss_by_region = combined_city_nfl_df['ratio']
    
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return population_by_region, win_loss_by_region, correlation

    #raleigh nc, charlotte nc

nfl_correlation()

(Metropolitan area
 Atlanta                    5789700.0
 Baltimore                  2798886.0
 Boston                     4794447.0
 Buffalo                    1132804.0
 Charlotte                  2474314.0
 Chicago                    9512999.0
 Cincinnati                 2165139.0
 Cleveland                  2055612.0
 Dallas–Fort Worth          7233323.0
 Denver                     2853077.0
 Detroit                    4297617.0
 Green Bay                   318236.0
 Houston                    6772470.0
 Indianapolis               2004230.0
 Jacksonville               1478212.0
 Kansas City                2104509.0
 Los Angeles               13310447.0
 Miami–Fort Lauderdale      6066387.0
 Minneapolis–Saint Paul     3551036.0
 Nashville                  1865298.0
 New Orleans                1268883.0
 New York City             20153634.0
 Philadelphia               6070500.0
 Phoenix                    4661537.0
 Pittsburgh                 2342299.0
 San Francisco Bay Area     665

In [9]:
population_by_region, win_loss_by_region, correlation = nfl_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"


In [10]:
## population_by_region, win_loss_by_region, correlation = nfl_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():

    mlb_df = pd.read_csv("assets/mlb.csv")
    nhl_df = pd.read_csv("assets/nhl.csv")
    nba_df = pd.read_csv("assets/nba.csv")
    nfl_df = pd.read_csv("assets/nfl.csv")
    cities_df = pd.read_html("assets/wikipedia_data.html",na_values='—')[1]
    cities_df = cities_df.iloc[:-1,[0,3,5,6,7,8]]    
    
    cities_df = cities_df.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True)
    cities_df.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities_df = cities_df.astype({'Population': 'int64'}) 
    cities_df = cities_df.replace('(\[note\s\d{1,10}\])', '', regex=True)
    cities_df = cities_df.replace('—[\s]?', " ", regex=True)
    #display(cities_df)
    

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values = sports_team_performance()
    
    #return p_values

sports_team_performance()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
p_values = sports_team_performance()

assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
